In [1]:
%%configure -f
{
    "conf": {
        "spark.jars.packages": "ai.catboost:catboost-spark_3.5_2.12:1.2.7",
        "spark.executor.memory": "24g",
        "spark.executor.cores": "4",       
        "spark.driver.memory": "24g",      
        "spark.yarn.am.memory": "4g",     
        "spark.dynamicAllocation.enabled": "true", 
        "spark.task.cpus": "4",          
        "spark.jars.packages.resolve.transitive": "true",
        "spark.executor.extraJavaOptions": "--add-exports java.base/sun.net.util=ALL-UNNAMED",
        "spark.driver.extraJavaOptions": "--add-exports java.base/sun.net.util=ALL-UNNAMED",
        "spark.network.timeout": "1200s",  
        "spark.rpc.askTimeout": "1200s", 
        "spark.executor.memoryOverhead": "4g"
    }
}

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.ml.linalg import Vectors
from pyspark.sql.types import StructType, StructField, DoubleType, StringType
import catboost_spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1735050777177_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Adding a parameter tag
cohort = 'cohort1'
age_band = '0-12'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import ai.catboost.spark as catboost_spark
import boto3
import json

# S3 Paths
s3_bucket = "s3://pgxdatalake"
ade_input_path = f"{s3_bucket}/apcd_ade_45/{age_band}"
non_ade_input_path = f"{s3_bucket}/apcd_non_ed/{age_band}"

# Read processed train and test datasets from S3
print("Reading train and test datasets...")
ade_df = spark.read.parquet(ade_input_path)
non_ade_df = spark.read.parquet(non_ade_input_path)

print("ADE and non-ADE datasets successfully loaded.")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Reading train and test datasets...
ADE and non-ADE datasets successfully loaded.

In [13]:
print("ADE DataFrame Schema:")
ade_df.printSchema()
print("\nnon-ADE DataFrame Schema:")
non_ade_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ADE DataFrame Schema:
root
 |-- mi_person_key: integer (nullable = true)
 |-- age_band: string (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- member_race: string (nullable = true)
 |-- payer_type: string (nullable = true)
 |-- payer_lob: string (nullable = true)
 |-- primary_insurance_flag: string (nullable = true)
 |-- member_zip_code_dos: string (nullable = true)
 |-- member_county_dos: string (nullable = true)
 |-- member_age_dos: integer (nullable = true)
 |-- ADE_Date: date (nullable = true)
 |-- drug_name: string (nullable = true)
 |-- drug_date: date (nullable = true)
 |-- drug_class_1: string (nullable = true)
 |-- drug_class_2: string (nullable = true)
 |-- drug_class_3: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Hospitalization: string (nullable = false)


non-ADE DataFrame Schema:
root
 |-- mi_person_key: integer (nullable = true)
 |-- age_band: string (nullable = true)
 |-- ADE_Date: date (nullable = true)
 |-- drug_name: string 

In [14]:
ade_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------+-------------+-----------+--------------------+----------+----------------------+-------------------+--------------------+--------------+----------+--------------------+----------+---------------+--------------------+--------------------+----+---------------+
|mi_person_key|age_band|member_gender|member_race|          payer_type| payer_lob|primary_insurance_flag|member_zip_code_dos|   member_county_dos|member_age_dos|  ADE_Date|           drug_name| drug_date|   drug_class_1|        drug_class_2|        drug_class_3|Year|Hospitalization|
+-------------+--------+-------------+-----------+--------------------+----------+----------------------+-------------------+--------------------+--------------+----------+--------------------+----------+---------------+--------------------+--------------------+----+---------------+
|   1000091911|    0-12|            M|         03|MLI - Medicaid Lo...|  MEDICAID|                     Y|              22031|       059 - FAIRFAX|  

In [15]:
non_ade_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------+----------+--------------------+----------+-----------------+--------------------+--------------------+----+---------------+
|mi_person_key|age_band|  ADE_Date|           drug_name| drug_date|     drug_class_1|        drug_class_2|        drug_class_3|Year|Hospitalization|
+-------------+--------+----------+--------------------+----------+-----------------+--------------------+--------------------+----+---------------+
|   1000002320|    0-12|2020-02-03|        erythromycin|2020-02-03|OPHTHALMIC AGENTS|Ophthalmic Anti-i...|Ophthalmic Antibi...|2020|              0|
|   1000005224|    0-12|2020-02-04|citalopram hydrob...|2019-12-27|  ANTIDEPRESSANTS|Selective Seroton...|Selective Seroton...|2020|              0|
|   1000005224|    0-12|2020-03-02|citalopram hydrob...|2020-02-17|  ANTIDEPRESSANTS|Selective Seroton...|Selective Seroton...|2020|              0|
|   1000005224|    0-12|2020-04-21|citalopram hydrob...|2020-03-12|  ANTIDEPRESSANTS|Selective Seroton...|

In [137]:

ade_df = ade_df.select("mi_person_key", "age_band", "ADE_Date", "drug_name", "drug_date", "Year", "Hospitalization")

non_ade_df = non_ade_df.select("mi_person_key", "age_band", "ADE_Date", "drug_name", "drug_date", "Year", "Hospitalization")

# Combine the two DataFrames
combined_df = ade_df.union(non_ade_df)

# Select the relevant columns for the model
train_df = combined_df.select('mi_person_key', 'drug_date', 'ADE_Date', 'drug_name', 'Year', 'Hospitalization')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [138]:
train_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- mi_person_key: integer (nullable = true)
 |-- drug_date: date (nullable = true)
 |-- ADE_Date: date (nullable = true)
 |-- drug_name: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Hospitalization: string (nullable = false)

In [139]:
from pyspark.sql.functions import col

unique_levels = train_df.groupBy("drug_name").count()
unique_levels.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+
|           drug_name| count|
+--------------------+------+
|compact space cha...|  2679|
|    dexcom g6 sensor|  1281|
|      precision xtra|  1347|
|onetouch delica p...|  2272|
|hydrocortisone ma...|   920|
|   gnp athletes foot|   261|
|       aquaphil base|     6|
|culturelle digest...|  4285|
|flintstones gummi...|   709|
|ranitidine hydroc...| 22956|
|  potassium chloride|  6483|
|cvs advanced 3-in...|    54|
|gnp fluticasone p...|   209|
|penicillin v pota...| 41798|
|           tretinoin| 12530|
|ethambutol hydroc...|   259|
|      ergocalciferol|  5194|
|      sertraline hcl|329458|
|cvs ibuprofen jun...|  1252|
|cetirizine hydroc...|607289|
+--------------------+------+
only showing top 20 rows

In [140]:
from pyspark.sql.functions import col, lower, regexp_replace, when, expr

# Function to alphabetically sort drug combinations
def sort_combination(drug_name):
    parts = drug_name.split('+')
    return '+'.join(sorted(parts))

# Register the UDF
spark.udf.register("sort_combination", sort_combination, StringType())

# Standardize drug names
clean_df = train_df.withColumn(
    'standardized_drug_name',
    lower(col('drug_name'))
)

# Remove trailing slashes using regexp_replace
clean_df = clean_df.withColumn('standardized_drug_name', 
    regexp_replace(col('standardized_drug_name'), '/$', ''))

# Replace spaces with underscores
clean_df = clean_df.withColumn(
    'standardized_drug_name',
    regexp_replace(col('standardized_drug_name'), ' ', '_')
)

# Replace '/' with '+'
clean_df = clean_df.withColumn(
    'standardized_drug_name',
    regexp_replace(col('standardized_drug_name'), '/', '+')
)

# Sort combinations if they contain '+'
clean_df = clean_df.withColumn(
    'standardized_drug_name',
    when(col('standardized_drug_name').contains('+'),
         expr('sort_combination(standardized_drug_name)'))
    .otherwise(col('standardized_drug_name'))
)

# Handle Non Drug Items
medical_supplies = [
    'accu-chek', 'knee_brace', 'lancet', 'syringe', 'needle', 'test_strip', 'monitor',
    'lancing_device', 'insulin_pump', 'glucose_meter', 'blood_glucose', 'nebulizer',
    'inhaler', 'spacer', 'chamber', 'compressor', 'catheter', 'dressing', 'bandage',
    'gauze', 'tape', 'alcohol_prep', 'alcohol_swab', 'pen_needle', 'aerochamber', 'onetouch',
    'optichamber'
]

# Filter out medical supplies and mark as 'not_drug'
for item in medical_supplies:
    clean_df = clean_df.withColumn('standardized_drug_name',
        when(col('standardized_drug_name').contains(item), 'not_drug')
        .otherwise(col('standardized_drug_name'))
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [141]:
from pyspark.sql.functions import create_map, lit, coalesce, col

# Function to apply a mapping
def replace_values_with_mapping(df, column_name, mapping):
    mapping_expr = create_map([lit(x) for pair in mapping.items() for x in pair])
    return df.withColumn(column_name, coalesce(mapping_expr[col(column_name)], col(column_name)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [142]:
# Consolidations

consolidations_a = {
    'thioridazine_hcl': 'thioridazine',
    'miconazole_nitrate': 'miconazole',
    'almotriptan_malate': 'almotriptan',
    'alogliptin/metformin_hcl': 'alogliptin+metformin',
    'alogliptin/pioglitazone': 'alogliptin+pioglitazone',
    'cvs_nicotine_polacrilex': 'nicotine',
    'metformin_hydrochloride': 'metformin',
    'rizatriptan_benzoate_odt': 'rizatriptan',
    'warfarin_sodium': 'warfarin',
    'buprenorphine_hcl': 'buprenorphine',
    'butalbital+acetaminophen': 'acetaminophen+butalbital',
    'cvs_acetaminophen': 'acetaminophen',
    'zolpidem_tartrate': 'zolpidem',
    'acetaminophen_childrens': 'acetaminophen+childrens',
    'acetaminophen_er': 'acetaminophen+er',
    'acetaminophen_extra_stren': 'acetaminophen+extra_strength',
    'acetaminophen_pm_extra_st': 'acetaminophen+pm+extra_strength',
    'acetaminophen+caffeine+di': 'acetaminophen+caffeine+diphenhydramine',
    'acetaminophen+codeine': 'acetaminophen+codeine',
    'acetaminophen+codeine_pho': 'acetaminophen+codeine+phosphate',
    'acetaminophen+diphenhydra': 'acetaminophen+diphenhydramine',
    'acebutolol_hcl': 'acebutolol',
    'acebutolol_hydrochloride': 'acebutolol',
    'acebutolol+hydrochloride': 'acebutolol',
    'acetazolamide_er': 'acetazolamide+er',
    'acetic_acid_0.25%': 'acetic_acid',
    'acetic_acid+aluminum_acet': 'acetic_acid+aluminum_acetate',
    'acetic_acid+hydrocortison': 'acetic_acid+hydrocortisone',
    
    # Aspirin
    'acetylsalicylic_acid': 'acetylsalicylic_acid',
    'acetylsalicylic_acid_81': 'acetylsalicylic_acid',
    'acetylsalicylic_acid_81_low_dose': 'acetylsalicylic_acid',
    'acetylsalicylic_acid_adult': 'acetylsalicylic_acid',
    'acetylsalicylic_acid_adult_low_dose': 'acetylsalicylic_acid',
    'acetylsalicylic_acid_adult_low_strengt': 'acetylsalicylic_acid',
    'acetylsalicylic_acid_childrens': 'acetylsalicylic_acid+childrens',
    'acetylsalicylic_acid_ec': 'acetylsalicylic_acid+ec',
    'acetylsalicylic_acid_ec_lo-dose': 'acetylsalicylic_acid+ec',
    'acetylsalicylic_acid_ec_low_dose': 'acetylsalicylic_acid+ec',
    'acetylsalicylic_acid_enteric_coated_ad': 'acetylsalicylic_acid+ec',
    'acetylsalicylic_acid_low_dose': 'acetylsalicylic_acid',
    'acetylsalicylic_acid_low_strength': 'acetylsalicylic_acid',
    'acetylsalicylic_acid_regular_strength': 'acetylsalicylic_acid',
    'acetylsalicylic_acidir-low': 'acetylsalicylic_acid',
    'acetylsalicylic_acid+dipyridamole': 'acetylsalicylic_acid+dipyridamole',
    'acetylsalicylic_acid+dipyridamole_er': 'acetylsalicylic_acid+dipyridamole+er',
    'acetylsalicylic_acidercreme': 'acetylsalicylic_acid+topical',
    'acetylsalicylic_acidercreme+aloe': 'acetylsalicylic_acid+aloe+topical',
    'acetylsalicylic_acidercreme_lidocaine': 'acetylsalicylic_acid+lidocaine+topical',
    'acetylsalicylic_acidercreme_lidocaine_max': 'acetylsalicylic_acid+lidocaine+max+topical',
    'acetylsalicylic_acidercreme_lidocaine_patc': 'acetylsalicylic_acid+lidocaine+patch+topical',
    'acetylsalicylic_acidercreme_max_roll-on_ar': 'acetylsalicylic_acid+max+roll_on+topical',
    'acetylsalicylic_acidercreme_w+lidocaine': 'acetylsalicylic_acid+lidocaine+topical',
    'aspir-low': 'acetylsalicylic_acid',
    'aspirin': 'acetylsalicylic_acid',
    'aspirin_81': 'acetylsalicylic_acid',
    'aspirin_81_low_dose': 'acetylsalicylic_acid',
    'aspirin_adult': 'acetylsalicylic_acid',
    'aspirin_adult_low_dose': 'acetylsalicylic_acid',
    'aspirin_adult_low_strengt': 'acetylsalicylic_acid',
    'aspirin_child': 'acetylsalicylic_acid+childrens',
    'aspirin_childrens': 'acetylsalicylic_acid+childrens',
    'aspirin_ec': 'acetylsalicylic_acid+ec',
    'aspirin_ec_lo-dose': 'acetylsalicylic_acid+ec',
    'aspirin_ec_low_dose': 'acetylsalicylic_acid+ec',
    'aspirin_enteric_coated_ad': 'acetylsalicylic_acid+ec',
    'aspirin_low_dose': 'acetylsalicylic_acid',
    'aspirin_low_strength': 'acetylsalicylic_acid',
    'aspirin_regular_strength': 'acetylsalicylic_acid',
    'aspirin+dipyridamole': 'acetylsalicylic_acid+dipyridamole',
    'aspirin+dipyridamole_er': 'acetylsalicylic_acid+dipyridamole+er',
    
    # Afluria (Seasonal)
    'afluria_2014-2015': 'afluria',
    'afluria_2016-2017': 'afluria',
    'afluria_2017-2018': 'afluria',

    # Afluria PF (Preservative-Free)
    'afluria_pf_2015-2016': 'afluria_pf',
    'afluria_pf_2016-2017': 'afluria_pf',
    'afluria_pf_2017-2018': 'afluria_pf',
    'afluria_pf_2018-2019': 'afluria_pf',

    # Afluria Quadrivalent
    'afluria_quadrivalent_2017': 'afluria_quadrivalent',
    'afluria_quadrivalent_2018': 'afluria_quadrivalent',
    'afluria_quadrivalent_2019': 'afluria_quadrivalent',
    'afluria_quadrivalent_2020': 'afluria_quadrivalent',
    
    # Albuterol
    'albuterol_sulfate': 'albuterol',
    'albuterol_sulfate_er': 'albuterol+er',
    'albuterol_sulfate_hfa': 'albuterol+hfa',
    
    # Additional 'A' drugs
    'allegra-d_12_hour': 'allegra-d+12_hour',
    'allegra-d_12_hour_allergen': 'allegra-d+12_hour',
    'allegra-d_24_hour': 'allegra-d+24_hour',
    'allegra-d_24_hour_allergen': 'allegra-d+24_hour',
    'allegra_allergen': 'allegra',
    'allegra_allergen_childrens': 'allegra+childrens',
    'afrin_sinus': 'afrin',
    'afrin_12_hour': 'afrin',
    'afrin_menthol': 'afrin+menthol',
    'afrin_nasal_spray': 'afrin',
    'afrin_nodrip_extra_moistu': 'afrin+nodrip+extra_moisture',
    'afrin_nodrip_severe_conge': 'afrin+nodrip+severe_congestion',
    'afrin_nodrip_sinus': 'afrin+nodrip+sinus',
    'afrin_pump_mist': 'afrin',
    'agamatrix_presto': 'not_drug',
    'agamatrix_ultra-thin_lanc': 'not_drug',
    'aimovig': 'aimovig',
    'airduo_respiclick_232/14': 'airduo_respiclick',
    'airial_compact_mini_nebul': 'not_drug',
    'airial_pediatric_nebulize': 'not_drug',
    'airs_disposable_nebulizer': 'not_drug',
    'alavert_allergen/sinus': 'alavert',
    'alaway': 'alaway',
    'alaway_childrens_allergen': 'alaway+childrens',
    'albuterol': 'albuterol',
    'albuterol_sulfate': 'albuterol',
    'albuterol_sulfate_er': 'albuterol+er',
    'albuterol_sulfate_hfa': 'albuterol+hfa',
    'alcohol_pads': 'not_drug',
    'alcohol_prep_pads': 'not_drug',
    'alcohol_preps': 'not_drug',
    'alcohol_swabs': 'not_drug',
    'alcortin_a': 'alcortin_a',
    'alendronate_sodium': 'alendronate',
    'alfuzosin_hcl_er': 'alfuzosin+er',
    'alive_womens_50+': 'alive_womens+50',
    'alive_womens_energy': 'alive_womens+energy',
    'alive_womens_gummy_vitami': 'alive_womens+gummy_vitamin',
    'all_day_allergen': 'allergy_relief',
    'all_day_allergen-d': 'allergy_relief',
    'all_day_allergen_d-12': 'allergy_relief',
    'all_day_pain_relief': 'all_day_pain_relief',
    'all_day_relief': 'all_day_relief',
    'allegra-d_12_hour': 'allergy_relief',
    'allegra-d_12_hour_allergen': 'allergy_relief',
    'allegra-d_24_hour_allergen': 'allergy_relief',
    'allegra_allergen': 'allergy_relief',
    'allegra_allergen_childrens': 'allergy_relief',
    'allergen_relief': 'allergy_relief',
    'allergen_relief/indoor/out': 'allergy_relief',
    'allergen_relief/nasal_deco': 'allergy_relief',
    'allergen_relief_24hr': 'allergy_relief',
    'allergen_relief_child': 'allergy_relief',
    'allergen_relief_d-24': 'allergy_relief',
    'allergy_relief': 'allergy_relief',
    'allergy_relief_24hr': 'allergy_relief',
    'allergy_relief_child': 'allergy_relief',
    'allergy_relief_d-24': 'allergy_relief',
    'allergy_relief/indoor/out': 'allergy_relief',
    'allergy_relief/nasal_deco': 'allergy_relief',
    'allergy_decongestant': 'allergy_relief',
    'alpha-lipoic_acid': 'alpha_lipoic_acid',
    'alpha_lipoic_acid': 'alpha_lipoic_acid',
    'alpha_lipoic_acid_extra_s': 'alpha_lipoic_acid',
    'alprazolam_er': 'alprazolam',
    'alprazolam_intensol': 'alprazolam',
    'alprazolam_odt': 'alprazolam',
    'alprazolam_xr': 'alprazolam',
    'alyacen_1/35': 'alyacen',
    'alyacen_7/7/7': 'alyacen',
    'amantadine_hcl': 'amantadine',
    'amantadine_hydrochloride': 'amantadine',
    'aspercreme_lidocaine_max': 'aspercreme+lidocaine',
    'aspercreme_lidocaine_patc': 'aspercreme+lidocaine',
    'aspercreme_max_roll-on_ar': 'aspercreme',
    'aspercreme_w+lidocaine': 'aspercreme+lidocaine',
    'aspercreme+aloe': 'aspercreme+aloe'

}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [143]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_a)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [144]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'a'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("a"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+---------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|      drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+---------------+----+---------------+----------------------+
|   1001020069|2020-01-06|2020-02-03|a+d diaper rash|2020|              0|       a+d_diaper_rash|
|   1009087144|2018-08-02|2018-08-02|a+d diaper rash|2018|              0|       a+d_diaper_rash|
|   1009087144|2018-08-02|2018-08-07|a+d diaper rash|2018|              0|       a+d_diaper_rash|
|   1009616212|2016-10-28|2016-10-28|  a+d first aid|2016|              0|         a+d_first_aid|
|   1013261304|2020-07-17|2020-08-03|  a+d first aid|2020|              0|         a+d_first_aid|
|   1009616212|2016-10-28|2016-10-29|  a+d first aid|2016|              0|         a+d_first_aid|
|   1006099180|2020-04-09|2020-05-04|    a+d prevent|2020|              0|           a+d_prevent|
|   1005248102|2018-

In [145]:
# B drug names
consolidations_b = {
    'b-12': 'vitamin_b12',
    'b-12_tr': 'vitamin_b12',
    'bacitracin_zinc': 'bacitracin',
    'bacitracin+polymyxin_b': 'bacitracin+polymyxin_b',
    'baclofen': 'baclofen',
    'bactrim': 'sulfamethoxazole+trimethoprim',
    'bactrim_ds': 'sulfamethoxazole+trimethoprim',
    'bactroban': 'mupirocin',
    'bactroban_nasal': 'mupirocin',
    'balcoltra': 'balcoltra',
    'balsalazide_disodium': 'balsalazide',
    'banophen': 'diphenhydramine',
    'baqsimi_one_pack': 'glucagon',
    'baqsimi_two_pack': 'glucagon',
    'basaglar_kwikpen': 'insulin_glargine',
    'bayer_microlet_lancets': 'not_drug',
    'beclomethasone_dipropiona': 'beclomethasone',
    'beconase_aq': 'beclomethasone',
    'belbuca': 'buprenorphine',
    'belladonna_alkaloids/phen': 'belladonna+phenobarbital',
    'belsomra': 'suvorexant',
    'benazepril': 'benazepril',
    'benazepril+hydrochlorothiazide': 'benazepril+hydrochlorothiazide',
    'benicar': 'olmesartan',
    'benicar_hct': 'olmesartan+hydrochlorothiazide',
    'benlysta': 'belimumab',
    'benztropine_mesylate': 'benztropine',
    'bepreve': 'bepotastine',
    'besivance': 'besifloxacin',
    'betamethasone_dipropionat': 'betamethasone',
    'betamethasone_valerate': 'betamethasone',
    'betaseron': 'interferon_beta-1b',
    'bethanechol_chloride': 'bethanechol',
    'bevespi_aerosphere': 'glycopyrrolate+formoterol',
    'bexsero': 'meningococcal_group_b_vaccine',
    'bicalutamide': 'bicalutamide',
    'bidil': 'isosorbide_dinitrate+hydralazine',
    'biktarvy': 'bictegravir+emtricitabine+tenofovir_alafenamide',
    'bimatoprost': 'bimatoprost',
    'bisacodyl_ec': 'bisacodyl',
    'bisoprolol_fumarate': 'bisoprolol',
    'bisoprolol_fumarate/hydro': 'bisoprolol+hydrochlorothiazide',
    'blisovi_24_fe': 'ethinyl_estradiol+norethindrone+iron',
    'blisovi_fe_1.5/30': 'ethinyl_estradiol+norethindrone+iron',
    'blisovi_fe_1/20': 'ethinyl_estradiol+norethindrone+iron',
    'boostrix': 'tetanus+diphtheria+pertussis_vaccine',
    'bosulif': 'bosutinib',
    'botox': 'onabotulinumtoxina',
    'bp_wash': 'benzoyl_peroxide',
    'braftovi': 'encorafenib',
    'breo_ellipta': 'fluticasone+vilanterol',
    'brilinta': 'ticagrelor',
    'brimonidine_tartrate': 'brimonidine',
    'briviact': 'brivaracetam',
    'bromfed_dm': 'brompheniramine+dextromethorphan+pseudoephedrine',
    'bromocriptine_mesylate': 'bromocriptine',
    'bromphen/pseudoephedrine': 'brompheniramine+pseudoephedrine',
    'brovana': 'arformoterol',
    'budesonide': 'budesonide',
    'budesonide_er': 'budesonide',
    'budesonide_nasal_spray': 'budesonide',
    'budesonide+formoterol_fum': 'budesonide+formoterol',
    'bumetanide': 'bumetanide',
    'bupap': 'butalbital+acetaminophen',
    'bupivacaine': 'bupivacaine',
    'buprenorphine': 'buprenorphine',
    'buprenorphine_hcl': 'buprenorphine',
    'buprenorphine_hcl/naloxon': 'buprenorphine+naloxone',
    'bupropion': 'bupropion',
    'bupropion_hcl': 'bupropion',
    'bupropion_hcl_xl': 'bupropion',
    'bupropion_hydrochloride': 'bupropion',
    'bupropion_hydrochloride_e': 'bupropion',
    'buspirone_hcl': 'buspirone',
    'buspirone_hydrochloride': 'buspirone',
    'butalbital+acetaminophen': 'butalbital+acetaminophen',
    'butorphanol_tartrate': 'butorphanol',
    'butrans': 'buprenorphine',
    'bydureon': 'exenatide',
    'bydureon_bcise': 'exenatide',
    'bydureon_pen': 'exenatide',
    'byetta': 'exenatide',
    'bystolic': 'nebivolol'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [146]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_b)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [147]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'b'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("b"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+---------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+---------+----+---------------+----------------------+
|   1003991503|2020-02-07|2020-03-09|      b-1|2020|              0|                   b-1|
|   1018166225|2020-08-03|2020-08-27|      b-1|2020|              0|                   b-1|
|   1011620421|2019-07-08|2019-07-09|      b-1|2019|              0|                   b-1|
|   1018166225|2020-08-03|2020-09-17|      b-1|2020|              0|                   b-1|
|   1003991503|2020-02-07|2020-03-11|      b-1|2020|              0|                   b-1|
|   1018166225|2020-08-03|2020-08-10|      b-1|2020|              0|                   b-1|
|   1011620421|2019-07-08|2019-07-27|      b-1|2019|              0|                   b-1|
|   1018166225|2020-08-03|2020-08-17|      b-1|2020|              0|            

In [148]:
# C Drug names
consolidations_c = {
    'cefazolin_sodium+dextrose': 'cefazolin+dextrose',
    'clindamycin+sodium_chlori': 'clindamycin+sodium_chloride',
    'cvs_b6': 'vitamin_b6',
    'cvs_digestive_probiotic': 'probiotic',
    'claritin': 'loratadine',
    'cvs_allergen': 'cetirizine',
    'centrum_men': 'multivitamin+men',
    'c-500': 'vitamin_c',
    'celexa': 'citalopram',
    'clindamycin_hcl': 'clindamycin',
    'culturelle_digestive_heal': 'probiotic',
    'cvs_spectravite_ultra_men': 'multivitamin+men',
    'cvs_allergy_eye_drops': 'allergy_eye_drops',
    'cvs_allergy': 'allergy_relief',
    'cvs_cold_&hot_maximum_st': 'pain_relief+hot_cold_therapy',
    'cvs_motion_sickness_relie': 'motion_sickness_relief',
    'caresens_n_blood_glucose': 'not_drug',
    'cvs_antiseptic_skin_clean': 'antiseptic_skin_cleanser',
    'clocortolone_pivalate_pum': 'clocortolone',
    'copper_caps': 'copper',
    'classic_prenatal': 'prenatal_vitamin',
    'caziant': 'caziant',
    'cvs_womens_daily_gummies': 'multivitamin+women',
    'cyclosporine_a': 'cyclosporine',
    'calcium_500+d3': 'calcium+vitamin_d',
    'concept_ob': 'prenatal_vitamin',
    'cvs_tension_headache': 'acetaminophen+caffeine',
    'cvs_petroleum_jelly': 'petroleum_jelly',
    'cepacol_sore_throat_+cou': 'cepacol',
    'corvite': 'multivitamin+iron',
    'cvs_heartburn_relief': 'heartburn_relief',
    'cvs_ibuprofen_ib': 'ibuprofen',
    'cytotec': 'misoprostol',
    'cvs_triple_antibiotic': 'triple_antibiotic',
    'cvs_blood_pressure_cuff': 'not_drug',
    'clobetasol_propionate': 'clobetasol',
    'cholestyramine_light': 'cholestyramine',
    'cimzia': 'certolizumab',
    'ciclopirox_nail_lacquer': 'ciclopirox',
    'cyproheptadine_hydrochlor': 'cyproheptadine',
    'cinacalcet_hydrochloride': 'cinacalcet',
    'cetirizine_hydrochloride': 'cetirizine',
    'ciprofloxacin_hydrochlori': 'ciprofloxacin',
    'cefuroxime_axetil': 'cefuroxime',
    'calcium+vitamin_d3': 'calcium+vitamin_d',
    'calcitonin_salmon': 'calcitonin',
    'calcium_plus_vitamin_d3': 'calcium+vitamin_d',
    'clomipramine_hydrochlorid': 'clomipramine',
    'calcium_600+_d': 'calcium+vitamin_d',
    'coly-mycin_s': 'colistin+neomycin+hydrocortisone',
    'cyclobenzaprine_hcl': 'cyclobenzaprine',
    'cvs_clotrimazole_3': 'clotrimazole',
    'cvs_antifungal_maxiumum_s': 'antifungal',
    'castellani_paint_modified': 'castellani_paint',
    'culturelle_ultimate_stren': 'probiotic',
    'cvs_selenium': 'selenium',
    'cvs_athletes_foot_powder': 'antifungal',
    'cvs_b-6': 'vitamin_b6',
    'cvs_8_hour_pain_relief': 'acetaminophen',
    'cvs_anti-dandruff': 'anti_dandruff_shampoo',
    'centrum_multigummies_adul': 'multivitamin',
    'cvs_spectravite_adult_gum': 'multivitamin',
    'cvs_sleep_aid_nighttime+m': 'sleep_aid',
    'ciprofloxacin_er': 'ciprofloxacin',
    'cvs_nighttime_tussin_dm': 'dextromethorphan+guaifenesin',
    'cholecalciferol': 'vitamin_d3',
    'c_250': 'vitamin_c',
    'clobetasol_propionate': 'clobetasol'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [149]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_c)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [150]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'c'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("c"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+--------------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|           drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+--------------------+----+---------------+----------------------+
|   1000674383|2018-11-20|2018-11-28|         calcium/c/d|2018|              0|           c+calcium+d|
|   1016077565|2017-09-29|2017-11-06|chewable vite wit...|2017|              0|  c+chewable_vite_w...|
|   1003455471|2018-04-04|2018-05-04|chewable vite wit...|2018|              0|  c+chewable_vite_w...|
|   1016077565|2017-09-29|2017-10-19|chewable vite wit...|2017|              0|  c+chewable_vite_w...|
|   1002759442|2018-08-09|2018-08-09|chewable vite wit...|2018|              0|  c+chewable_vite_w...|
|   1016077565|2017-09-29|2017-10-17|chewable vite wit...|2017|              0|  c+chewable_vite_w...|
|   1003455471|2018-05-16|2018-05-16|chewable vite wit...|2018|          

In [151]:
# D drug names

consolidations_d = {
    'dexmethylphenidate_hcl_er': 'dexmethylphenidate_er',
    'dialyvite_800': 'dialyvite',
    'dialyvite_800/zinc_15': 'dialyvite+zinc',
    'diclofenac_sodium': 'diclofenac',
    'diclofenac_epolamine': 'diclofenac',
    'diltiazem_hcl_er': 'diltiazem+er',
    'diltiazem_hydrochloride_e': 'diltiazem+er',
    'dimethyl_fumarate_starter': 'dimethyl_fumarate',
    'diphenoxylate/atropine': 'diphenoxylate+atropine',
    'disopyramide_phosphate': 'disopyramide',
    'divigel': 'estradiol_gel',
    'divalproex_sodium_dr': 'divalproex',
    'docusate_sodium/senna': 'docusate+senna',
    'dobutamine_hydrochloride': 'dobutamine',
    'dologesic': 'acetaminophen+caffeine',
    'donnatal': 'phenobarbital+belladonna_alkaloids',
    'dorzolamide_hydrochloride': 'dorzolamide',
    'doxazosin_mesylate': 'doxazosin',
    'doxylamine_succinate/pyri': 'doxylamine+pyridoxine',
    'dritho-creme_hp': 'anthralin',
    'duloxetine_hydrochloride': 'duloxetine',
    'dymista': 'azelastine+fluticasone',
    'dynabac_5.0': 'dirithromycin'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [152]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_d)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [153]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'd'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("d"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+--------------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|           drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+--------------------+----+---------------+----------------------+
|   1001104270|2020-10-29|2020-10-29|eq calcium 600+d+...|2020|              0|  d+eq_calcium_600+...|
|   1014896941|2020-05-13|2020-05-20|homeneb with side...|2020|              0|  d+homeneb_with_si...|
|   1000117761|2019-03-23|2019-03-23|homeneb with side...|2019|              1|  d+homeneb_with_si...|
|   1014896941|2020-05-13|2020-05-13|homeneb with side...|2020|              0|  d+homeneb_with_si...|
|   1006957984|2020-08-05|2020-09-09|homeneb with side...|2020|              0|  d+homeneb_with_si...|
|   1014896941|2020-05-13|2020-05-19|homeneb with side...|2020|              0|  d+homeneb_with_si...|
|   1007782392|2020-03-06|2020-04-15|homeneb with side...|2020|          

In [154]:
# E drug names

consolidations_e = {
    'edarbyclor': 'azilsartan+chlorthalidone',
    'eliquis_starter_pack': 'apixaban',
    'eq_mucus_relief_dm': 'guaifenesin+dextromethorphan',
    'euthyrox': 'levothyroxine',
    'epivir': 'lamivudine',
    'evotaz': 'atazanavir+cobicistat',
    'ethambutol_hydrochloride': 'ethambutol',
    'enalapril_maleate': 'enalapril',
    'estradiol_valerate': 'estradiol',
    'ergocalciferol': 'vitamin_d2',
    'estrace': 'estradiol',
    'estring': 'estradiol',
    'erlotinib_hydrochloride': 'erlotinib',
    'ertapenem_sodium': 'ertapenem',
    'edex': 'alprostadil',
    'epipen': 'epinephrine',
    'eylea': 'aflibercept',
    'easy_touch_pen_needles_32': 'not_drug',
    'easy_touch_fliplock_safet': 'not_drug',
    'easy_touch_lancets_30g/pr': 'not_drug',
    'easy_touch_safety_lancets': 'not_drug',
    'eucerin_skin_calming_dail': 'not_drug',
    'eucerin_daily_protection/': 'not_drug'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [155]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_e)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [156]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'e'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("e"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+--------------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|           drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+--------------------+----+---------------+----------------------+
|   1015002705|2016-04-04|2016-04-04|gnp childrens che...|2016|              0|  e+gnp_childrens_c...|
|   1015794407|2020-04-16|2020-04-16|lidocaine hydroch...|2020|              0|  e+lidocaine_hydro...|
|   1008853099|2020-09-10|2020-09-10|lidocaine hydroch...|2020|              0|  e+lidocaine_hydro...|
|   1006486763|2020-06-12|2020-06-12|lidocaine hydroch...|2020|              0|  e+lidocaine_hydro...|
|   1000866199|2020-11-20|2020-11-20|lidocaine hydroch...|2020|              0|  e+lidocaine_hydro...|
|   1006486763|2020-06-12|2020-06-26|lidocaine hydroch...|2020|              0|  e+lidocaine_hydro...|
|   1005933259|2020-06-28|2020-06-28|lidocaine hydroch...|2020|          

In [157]:
# F drug names
consolidations_f = {
    'fioricet': 'butalbital+acetaminophen+caffeine',
    'fiorinal+codeine': 'butalbital+acetaminophen+codeine',
    'firazyr': 'icatibant',
    'firmagon': 'degarelix',
    'flagyl': 'metronidazole',
    'flagyl_er': 'metronidazole_er',
    'flecainide': 'flecainide',
    'flomax': 'tamsulosin',
    'flonase': 'fluticasone',
    'flovent': 'fluticasone',
    'focalin': 'dexmethylphenidate',
    'forteo': 'teriparatide',
    'fosamax': 'alendronate',
    'furosemide': 'furosemide'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [158]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_f)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [159]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'f'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("f"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+-------------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|          drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+-------------------+----+---------------+----------------------+
|   1002394546|2018-06-12|2018-06-25|g-levocarnitine s/f|2018|              0|   f+g-levocarnitine_s|
|   1002394546|2018-06-12|2018-06-12|g-levocarnitine s/f|2018|              0|   f+g-levocarnitine_s|
|   1002394546|2018-06-12|2018-07-26|g-levocarnitine s/f|2018|              0|   f+g-levocarnitine_s|
|   1002394546|2018-06-12|2018-06-14|g-levocarnitine s/f|2018|              0|   f+g-levocarnitine_s|
|   1006009655|2020-10-05|2020-10-14|         nanovm t/f|2020|              0|            f+nanovm_t|
|   1006009655|2020-11-03|2020-12-09|         nanovm t/f|2020|              0|            f+nanovm_t|
|   1006009655|2020-09-01|2020-10-14|         nanovm t/f|2020|              0|    

In [160]:
# G drug names

consolidations_g = {
    'gentamicin_topical': 'gentamicin',
    'genvoya': 'elvitegravir+cobicistat+emtricitabine+tenofovir_alafenamide',
    'gilenya': 'fingolimod',
    'glatiramer_acetate': 'glatiramer',
    'glucophage': 'metformin',
    'glyxambi': 'empagliflozin+linagliptin',
    'gnp_mucus_er': 'guaifenesin',
    'gnp_acetylsalicylic_acidirin_low_dose': 'acetylsalicylic_acid',
    'gnp_acetylsalicylic_acidirin': 'acetylsalicylic_acid',
    'gnp_tussin_dm_cough': 'guaifenesin+dextromethorphan',
    'gnp_loratadine': 'loratadine',
    'gnp_lancets_thin_26g': 'not_drug',
    'glatopa': 'glatiramer',
    'glucotrol': 'glipizide',
    'glycolax': 'polyethylene_glycol_3350',
    'glyxambi': 'empagliflozin+linagliptin',
    'gocovri': 'amantadine',
    'golytely': 'polyethylene_glycol_3350+electrolytes',
    'gvoke_hypopen': 'glucagon',
    'guanfacine_hcl': 'guanfacine',
    'guanfacine_er': 'guanfacine+er',
    'guanfacine_hydrochloride': 'guanfacine'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [161]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_g)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [162]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'g'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("g"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+--------------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|           drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+--------------------+----+---------------+----------------------+
|   1002620014|2020-06-02|2020-06-09|g4 platinum recei...|2020|              0|  g4_platinum_recei...|
|   1002620014|2020-06-02|2020-07-03|g4 platinum recei...|2020|              0|  g4_platinum_recei...|
|   1002620014|2020-06-02|2020-07-09|g4 platinum recei...|2020|              0|  g4_platinum_recei...|
|   1009068435|2018-01-30|2018-02-13|         g tussin ac|2018|              0|           g_tussin_ac|
|   1019800590|2018-06-14|2018-07-05|         g tussin ac|2018|              0|           g_tussin_ac|
|   1009068435|2018-01-30|2018-02-20|         g tussin ac|2018|              0|           g_tussin_ac|
|   1006566906|2018-02-13|2018-02-13|         g tussin ac|2018|          

In [163]:
# H drug names

consolidations_h = {
    'harvoni': 'ledipasvir+sofosbuvir',
    'humira': 'adalimumab',
    'hydrocortisone+acetate+pramoxine': 'hydrocortisone+pramoxine',
    'hydrocodone_bitartrate_er': 'hydrocodone+er',
    'hydromorphone_hcl_er': 'hydromorphone+er',
    'hydroxychloroquine_sulfat': 'hydroxychloroquine',
    'humulin_70/30': 'insulin_nph',
    'humulin_70/30_kwikpen': 'insulin_nph',
    'humulin_n': 'insulin_nph',
    'humulin_n_kwikpen': 'insulin_nph',
    'humulin_r_u-500': 'insulin_regular',
    'halobetasol_propionate': 'halobetasol',
    'heparin_lock': 'heparin',
    'heparin_lock_flush+sodium_chloride': 'heparin+sodium_chloride',
    'humatrope_combo_pack': 'somatropin',
    'hurricaine_one': 'benzocaine',
    'hydramine': 'diphenhydramine',
    'hydroxypropyl_methylcellu': 'hydroxypropyl_methylcellulose',
    'h-chlor_12': 'chlorpheniramine',
    'hemangeol': 'propranolol'
}


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [164]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_h)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [165]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'h'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("h"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+--------------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|           drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+--------------------+----+---------------+----------------------+
|   1003621617|2017-02-10|2017-02-10|quintet glucose c...|2017|              0|  h+quintet_glucose...|
|   1003621617|2017-02-13|2017-03-02|quintet glucose c...|2017|              0|  h+quintet_glucose...|
|   1003621617|2017-02-10|2017-03-02|quintet glucose c...|2017|              0|  h+quintet_glucose...|
|   1003621617|2017-02-14|2017-03-02|quintet glucose c...|2017|              0|  h+quintet_glucose...|
|   1003621617|2017-02-10|2017-02-12|quintet glucose c...|2017|              0|  h+quintet_glucose...|
|   1003621617|2017-02-10|2017-02-11|quintet glucose c...|2017|              0|  h+quintet_glucose...|
|   1016906917|2020-10-30|2020-11-06|h-r lubricating j...|2020|          

In [166]:
# I drug names

consolidations_i = {
    'imodium': 'loperamide',
    'ibu': 'ibuprofen',
    'ibu-200': 'ibuprofen',
    'ibu-drops': 'ibuprofen',
    'ibu-drops_infants': 'ibuprofen',
    'insulin_aspart': 'insulin_aspart',
    'insulin_degludec': 'insulin_degludec',
    'insulin_detemir': 'insulin_detemir',
    'insulin_glargine': 'insulin_glargine',
    'insulin_lispro': 'insulin_lispro',
    'insulin_regular': 'insulin_regular',
    'ibrance': 'palbociclib',
    'iclusig': 'ponatinib',
    'idhifa': 'enasidenib',
    'ilumya': 'tildrakizumab',
    'imbruvica': 'ibrutinib',
    'imdur': 'isosorbide_mononitrate',
    'imfinzi': 'durvalumab',
    'imitrex': 'sumatriptan',
    'impoyz': 'clobetasol',
    'imuran': 'azathioprine',
    'inbrija': 'levodopa',
    'incruse_ellipta': 'umeclidinium',
    'inderal': 'propranolol',
    'indocin': 'indomethacin',
    'inflectra': 'infliximab',
    'ingrezza': 'valbenazine',
    'injectafer': 'ferric_carboxymaltose',
    'inlyta': 'axitinib',
    'inrebic': 'fedratinib',
    'intuniv': 'guanfacine',
    'invega': 'paliperidone',
    'invokana': 'canagliflozin'
}


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [167]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_i)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [168]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'i'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("i"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+--------------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|           drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+--------------------+----+---------------+----------------------+
|   1010249221|2020-06-01|2020-06-22|similac neosure a...|2020|              0|  i+similac_neosure...|
|   1008064837|2018-02-21|2018-04-03|similac neosure a...|2018|              0|  i+similac_neosure...|
|   1010249221|2020-06-01|2020-06-30|similac neosure a...|2020|              0|  i+similac_neosure...|
|   1008064837|2018-02-21|2018-04-05|similac neosure a...|2018|              0|  i+similac_neosure...|
|   1010249221|2020-06-01|2020-06-05|similac neosure a...|2020|              0|  i+similac_neosure...|
|   1008064837|2018-02-21|2018-03-16|similac neosure a...|2018|              0|  i+similac_neosure...|
|   1008064837|2018-02-21|2018-03-07|similac neosure a...|2018|          

In [169]:
# J drug names

consolidations_j = {
    'januvia': 'sitagliptin',
    'jornay_pm': 'methylphenidate',
    'jublia': 'efinaconazole',
    'junel_fe_24': 'ethinyl_estradiol+norethindrone+iron',
    'jardiance': 'empagliflozin',
    'jantoven': 'warfarin',
    'janumet': 'sitagliptin+metformin',
    'jakafi': 'ruxolitinib',
    'jadenu': 'deferasirox',
    'jentadueto': 'linagliptin+metformin',
    'jevtana': 'cabazitaxel',
    'jencycla': 'norethindrone',
    'juluca': 'dolutegravir+rilpivirine',
    'junel_fe_1/20': 'ethinyl_estradiol+norethindrone+iron',
    'junel_fe_1.5/30': 'ethinyl_estradiol+norethindrone+iron',
    'junel_1/20': 'ethinyl_estradiol+norethindrone',
    'junel_1.5/30': 'ethinyl_estradiol+norethindrone',
    'jinteli': 'ethinyl_estradiol+norethindrone'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [170]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_j)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [171]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'j'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("j"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+---------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|      drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+---------------+----+---------------+----------------------+
|   1011549721|2020-07-13|2020-07-28|jadenu sprinkle|2020|              0|       jadenu_sprinkle|
|   1000258842|2018-02-26|2018-04-07|jadenu sprinkle|2018|              0|       jadenu_sprinkle|
|   1002826350|2018-02-09|2018-03-22|jadenu sprinkle|2018|              0|       jadenu_sprinkle|
|   1000258842|2018-01-31|2018-02-14|jadenu sprinkle|2018|              0|       jadenu_sprinkle|
|   1000258842|2017-12-07|2017-12-29|jadenu sprinkle|2017|              0|       jadenu_sprinkle|
|   1000258842|2018-01-22|2018-02-13|jadenu sprinkle|2018|              0|       jadenu_sprinkle|
|   1002826350|2018-02-09|2018-02-25|jadenu sprinkle|2018|              0|       jadenu_sprinkle|
|   1000258842|2018-

In [172]:
# K drug names

consolidations_k = {
    'kaletra': 'lopinavir+ritonavir',
    'kcentra': 'prothrombin_complex_concentrate',
    'keflex': 'cephalexin',
    'keppra': 'levetiracetam',
    'keytruda': 'pembrolizumab',
    'kineret': 'anakinra',
    'kisqali': 'ribociclib',
    'klonopin': 'clonazepam',
    'klor-con': 'potassium_chloride',
    'kombiglyze_xr': 'metformin+saxagliptin',
    'kuvan': 'sapropterin',
    'kyleena': 'levonorgestrel'
}


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [173]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_k)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [174]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'k'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("k"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+---------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+---------+----+---------------+----------------------+
|   1006435739|2019-01-07|2019-01-18|   k-phos|2019|              0|                k-phos|
|   1006435739|2018-12-07|2018-12-21|   k-phos|2018|              0|                k-phos|
|   1000870130|2017-12-28|2018-02-05|   k-phos|2018|              0|                k-phos|
|   1006435739|2018-12-07|2018-12-19|   k-phos|2018|              0|                k-phos|
|   1010619946|2018-04-11|2018-05-02|   k-phos|2018|              0|                k-phos|
|   1006435739|2018-12-26|2018-12-29|   k-phos|2018|              0|                k-phos|
|   1000870130|2017-12-28|2018-01-04|   k-phos|2018|              0|                k-phos|
|   1006435739|2018-12-07|2018-12-29|   k-phos|2018|              0|            

In [175]:
# L drug names
consolidations_l = {
    'lovaza': 'omega-3-acid_ethyl_esters',
    'lucemyra': 'lofexidine',
    'lyrica': 'pregabalin',
    'levalbuterol_tartrate_hfa': 'levalbuterol',
    'lidocaine_hydrochloride': 'lidocaine',
    'lidocaine_5%': 'lidocaine',
    'livalo': 'pitavastatin',
    'lmx_5': 'lidocaine',
    'loestrin_fe_1.5/30': 'ethinyl_estradiol+norethindrone+iron',
    'loestrin_1/20-21': 'ethinyl_estradiol+norethindrone',
    'loteprednol_etabonate': 'loteprednol',
    'loxasperse': 'not_drug',
    'lysodren': 'mitotane'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [176]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_l)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [177]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'l'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("l"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+--------------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|           drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+--------------------+----+---------------+----------------------+
|   1000601369|2020-08-14|2020-09-10|l-5-methyltetrahy...|2020|              0|  l-5-methyltetrahy...|
|   1000601369|2020-08-17|2020-09-10|l-5-methyltetrahy...|2020|              0|  l-5-methyltetrahy...|
|   1000601369|2020-08-14|2020-09-15|l-5-methyltetrahy...|2020|              0|  l-5-methyltetrahy...|
|   1000601369|2020-08-17|2020-09-15|l-5-methyltetrahy...|2020|              0|  l-5-methyltetrahy...|
|   1000601369|2020-08-14|2020-09-22|l-5-methyltetrahy...|2020|              0|  l-5-methyltetrahy...|
|   1000601369|2020-08-17|2020-09-22|l-5-methyltetrahy...|2020|              0|  l-5-methyltetrahy...|
|   1000601369|2020-08-14|2020-09-24|l-5-methyltetrahy...|2020|          

In [178]:
# M drug names

consolidations_m = {
    'metformin_hydrochloride': 'metformin',
    'metformin_hydrochloride_e': 'metformin_er',
    'metoprolol_tartrate': 'metoprolol',
    'metformin_hcl': 'metformin',
    'methylprednisolone_dose_p': 'methylprednisolone',
    'memantine_hydrochloride': 'memantine',
    'metoclopramide_hcl': 'metoclopramide',
    'medroxyprogesterone_aceta': 'medroxyprogesterone',
    'metoprolol_succinate_er': 'metoprolol_er',
    'methylphenidate_hydrochlo': 'methylphenidate',
    'metformin_hcl_er': 'metformin_er',
    'montelukast_sodium': 'montelukast',
    'methotrexate': 'methotrexate',
    'metoprolol_tartrate': 'metoprolol',
    'methadone_hcl': 'methadone',
    'minocycline_hcl': 'minocycline',
    'metoprolol_succinate': 'metoprolol',
    'methenamine_hippurate': 'methenamine',
    'methylphenidate_hcl_er': 'methylphenidate_er',
    'metronidazole_vaginal': 'metronidazole',
    'metoprolol_tartrate/hydro': 'metoprolol+hydrochlorothiazide',
    'midodrine_hcl': 'midodrine',
    'metoclopramide_hydrochlo': 'metoclopramide',
    'meclizine_hcl': 'meclizine',
    'megestrol_acetate': 'megestrol',
    'metformin/sitagliptin': 'metformin+sitagliptin',
    'mycophenolate_mofetil': 'mycophenolate',
    'methadone_hydrochloride': 'methadone',
    'miconazole_nitrate': 'miconazole',
    'metronidazole_er': 'metronidazole+er',
    'methylphenidate_er': 'methylphenidate_er',
    'morphine_sulfate_er': 'morphine+er',
    'moxifloxacin_hcl': 'moxifloxacin',
    'metoprolol_er_succinate': 'metoprolol_er',
    'miralax': 'polyethylene_glycol_3350',
    'metamucil': 'psyllium',
    'mometasone_furoate': 'mometasone',
    'multivitamin/minerals': 'multivitamin+minerals'
}


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [179]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_m)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [180]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'm'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("m"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+--------------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|           drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+--------------------+----+---------------+----------------------+
|   1021721082|2019-12-10|2019-12-10|zarbees sleep chi...|2019|              0|  m+zarbees_sleep_c...|
|   1003741075|2020-05-07|2020-05-11|zarbees sleep chi...|2020|              0|  m+zarbees_sleep_c...|
|   1009842543|2019-11-14|2019-12-06|zarbees sleep chi...|2019|              0|  m+zarbees_sleep_c...|
|   1003741075|2020-05-07|2020-05-20|zarbees sleep chi...|2020|              0|  m+zarbees_sleep_c...|
|   1009842543|2019-11-14|2019-12-26|zarbees sleep chi...|2019|              0|  m+zarbees_sleep_c...|
|   1003741075|2020-05-07|2020-05-18|zarbees sleep chi...|2020|              0|  m+zarbees_sleep_c...|
|   1009842543|2019-11-14|2019-12-13|zarbees sleep chi...|2019|          

In [181]:
# N drug names

consolidations_n = {
    'namenda': 'memantine',
    'naprosyn': 'naproxen',
    'narcan': 'naloxone',
    'nasacort': 'triamcinolone',
    'nasonex': 'mometasone',
    'neosporin': 'bacitracin+neomycin+polymyxin_b',
    'neurontin': 'gabapentin',
    'nexium': 'esomeprazole',
    'nexium_24hr': 'esomeprazole',
    'nizoral_a-d': 'nizoral',
    'norco': 'hydrocodone+acetaminophen',
    'norvasc': 'amlodipine',
    'nortriptyline_hydrochlori': 'nortriptyline',
    'nortriptyline_hcl': 'nortriptyline',
    'novolin_n': 'insulin_nph',
    'novolog': 'insulin_aspart',
    'np_thyroid': 'thyroid',
    'nystatin_topical': 'nystatin',
    'novolin_70/30': 'insulin_nph',
    'novolin_70/30_flexpen': 'insulin_nph',
    'novolin_70/30_flexpen_rel': 'insulin_nph',
    'novolin_70/30_penfill': 'insulin_nph',
    'novolin_70/30_relion': 'insulin_nph',
    'novolin_n': 'insulin_nph',
    'novolin_n_flexpen': 'insulin_nph',
    'novolin_n_flexpen_relion': 'insulin_nph',
    'novolin_n_relion': 'insulin_nph',
    'novolin_r': 'insulin_regular',
    'novolin_r_flexpen_relion': 'insulin_regular',
    'novolin_r_relion': 'insulin_regular',
    'novolog': 'insulin_aspart',
    'novolog_flexpen': 'insulin_aspart',
    'novolog_mix_70/30': 'insulin_aspart+protamine',
    'novolog_mix_70/30_prefill': 'insulin_aspart+protamine',
    'novolog_penfill': 'insulin_aspart',
    'np_thyroid_120': 'np_thyroid',
    'np_thyroid_15': 'np_thyroid',
    'np_thyroid_30': 'np_thyroid',
    'np_thyroid_60': 'np_thyroid',
    'np_thyroid_90': 'np_thyroid',
    'nystatin_foreign': 'nystatin',
    'nystatin+triamcinolone_ac': 'nystatin+triamcinolone'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [182]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_n)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [183]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'n'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("n"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+--------------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|           drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+--------------------+----+---------------+----------------------+
|   1001832042|2019-06-06|2019-07-05|       n-a-c sustain|2019|              0|         n-a-c_sustain|
|   1001832042|2019-06-06|2019-07-15|       n-a-c sustain|2019|              0|         n-a-c_sustain|
|   1001832042|2019-06-06|2019-06-14|       n-a-c sustain|2019|              0|         n-a-c_sustain|
|   1001832042|2019-06-06|2019-07-12|       n-a-c sustain|2019|              0|         n-a-c_sustain|
|   1001832042|2019-06-06|2019-06-06|       n-a-c sustain|2019|              0|         n-a-c_sustain|
|   1001832042|2019-03-19|2019-03-22| n-acetyl-l-cysteine|2019|              0|   n-acetyl-l-cysteine|
|   1001832042|2019-03-19|2019-04-05| n-acetyl-l-cysteine|2019|          

In [184]:
# O drug names

consolidations_o = {
    'oxycodone_hydrochloride': 'oxycodone',
    'oxycodone+acetaminophen': 'oxycodone+acetaminophen',
    'ondansetron_hydrochloride': 'ondansetron',
    'ondansetron_hcl': 'ondansetron',
    'ondansetron_hcl_dihydrate': 'ondansetron',
    'ondansetron_odt': 'ondansetron+odt',
    'oxybutynin_chloride': 'oxybutynin',
    'oxybutynin_chloride_er': 'oxybutynin+er',
    'olmesartan_medoxomil': 'olmesartan',
    'omega-3-acid_ethyl_esters': 'omega-3-acid_ethyl_esters',
    'oseltamivir_phosphate': 'oseltamivir',
    'oxymorphone_hydrochloride': 'oxymorphone',
    'optivar': 'azelastine',
    'opsumit': 'macitentan',
    'ocaliva': 'obeticholic_acid',
    'octagam': 'immune_globulin_intravenous',
    'ocuflox': 'ofloxacin_ophthalmic',
    'ofev': 'nintedanib',
    'ogivri': 'trastuzumab',
    'olumiant': 'baricitinib',
    'omnaris': 'ciclesonide_nasal',
    'onfi': 'clobazam',
    'onglyza': 'saxagliptin',
    'onpattro': 'patisiran',
    'opana': 'oxymorphone',
    'opdivo': 'nivolumab',
    'opsumit': 'macitentan',
    'oracea': 'doxycycline',
    'oralair': 'grass_pollen_allergen_extract',
    'oravig': 'miconazole',
    'orencia': 'abatacept',
    'orfadin': 'nitisinone',
    'orkambi': 'lumacaftor+ivacaftor',
    'ortho_evra': 'ethinyl_estradiol+norelgestromin',
    'ortho_tri-cyclen': 'ethinyl_estradiol+norgestimate',
    'oseni': 'alogliptin+pioglitazone',
    'otezla': 'apremilast',
    'oxaydo': 'oxycodone',
    'oxtellar_xr': 'oxcarbazepine',
    'ozempic': 'semaglutide',
    'olanzapine_odt': 'olanzapine+odt',
    'olmesartan_medoxomil': 'olmesartan',
    'olmesartan_medoxomil+amlo': 'olmesartan+amlodipine',
    'olmesartan_medoxomil+hydr': 'olmesartan+hydrochlorothiazide',
    'omega-3-6-9': 'omega-3+6+9',
    'omega-3-acid_ethyl_esters': 'omega-3',
    'omega-3_2100': 'omega-3',
    'omega-3_cf': 'omega-3',
    'omega-3_epa_fish_oil': 'omega-3',
    'omega-3_fish_oil': 'omega-3',
    'omega-3_fish_oil_maximum': 'omega-3',
    'omega-3_fish_oil_no_burp': 'omega-3',
    'omega-3_krill_oil': 'omega-3+krill_oil',
    'omega-3_rx_complete': 'omega-3',
    'omega-3+d-3_wellness_pack': 'omega-3+vitamin_d3',
    'omega_3': 'omega-3',
    'omega_3-6-9_complex': 'omega-3+6+9',
    'omega_iii_epa+docosahexaenoic_acid': 'omega-3'
}


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [185]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_o)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [186]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'o'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("o"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+--------------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|           drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+--------------------+----+---------------+----------------------+
|   1000601369|2020-04-16|2020-05-19|oasis moisturizin...|2020|              0|  oasis_moisturizin...|
|   1000601369|2020-04-16|2020-05-01|oasis moisturizin...|2020|              0|  oasis_moisturizin...|
|   1000601369|2020-04-16|2020-05-29|oasis moisturizin...|2020|              0|  oasis_moisturizin...|
|   1000601369|2020-04-16|2020-05-22|oasis moisturizin...|2020|              0|  oasis_moisturizin...|
|   1000601369|2020-04-16|2020-05-15|oasis moisturizin...|2020|              0|  oasis_moisturizin...|
|   1000601369|2020-04-16|2020-05-07|oasis moisturizin...|2020|              0|  oasis_moisturizin...|
|   1000601369|2020-04-16|2020-05-08|oasis moisturizin...|2020|          

In [187]:
# P drug names

consolidations_p = {
    'pantoprazole_sodium': 'pantoprazole',
    'paroxetine_hcl': 'paroxetine',
    'paroxetine_hcl_er': 'paroxetine+er',
    'paroxetine_hydrochloride': 'paroxetine',
    'penicillin_v_potassium': 'penicillin',
    'penicillin_g_potassium': 'penicillin',
    'penicillin_g_sodium': 'penicillin',
    'phenytoin_sodium_extended': 'phenytoin',
    'pioglitazone_hcl': 'pioglitazone',
    'potassium_chloride_er': 'potassium_chloride+er',
    'pravastatin_sodium': 'pravastatin',
    'prednisolone_acetate': 'prednisolone',
    'prednisone_intensol': 'prednisone',
    'promethazine_hcl': 'promethazine',
    'propranolol_hcl': 'propranolol',
    'peg3350': 'polyethylene_glycol_3350',
    'peg_3350': 'polyethylene_glycol_3350',
    'pneumovax_23': 'pneumococcal_vaccine',
    'pradaxa': 'dabigatran',
    'proair_hfa': 'albuterol',
    'pyridium': 'phenazopyridine',
    'plavix': 'clopidogrel',
    'protonix': 'pantoprazole',
    'prozac': 'fluoxetine',
    'premarin': 'conjugated_estrogens',
    'prilosec': 'omeprazole',
    'prevacid': 'lansoprazole',
    'plaquenil': 'hydroxychloroquine',
    'paxil': 'paroxetine',
    'proventil_hfa': 'albuterol',
    'provera': 'medroxyprogesterone',
    'prograf': 'tacrolimus',
    'percocet': 'oxycodone+acetaminophen',
    'prempro': 'conjugated_estrogens+medroxyprogesterone',
    'procardia_xl': 'nifedipine',
    'provigil': 'modafinil',
    'prilosec_otc': 'omeprazole',
    'phenazopyridine_hcl': 'phenazopyridine',
    'phenazopyridine_hydrochlo': 'phenazopyridine',
    'phentermine_hcl': 'phentermine',
    'phentermine_hydrochloride': 'phentermine',
    'phentolamine_mesylate': 'phentolamine',
    'phenylephrine_hydrochlori': 'phenylephrine',
    'pilocarpine_hcl': 'pilocarpine',
    'pilocarpine_hydrochloride': 'pilocarpine',
    'pioglitazone_hcl': 'pioglitazone',
    'pioglitazone_hydrochlorid': 'pioglitazone',
    'pioglitazone_hcl+glimepir': 'pioglitazone+glimepiride',
    'pioglitazone_hcl+metformi': 'pioglitazone+metformin',
    'piperacillin_sodium+tazob': 'piperacillin+tazobactam',
    'piperacillin+tazobactam': 'piperacillin+tazobactam',
    'prazosin_hcl': 'prazosin',
    'prazosin_hydrochloride': 'prazosin',
    'peg-3350+electrolytes': 'polyethylene_glycol_3350+electrolytes',
    'peg-3350+electrolytes+asc': 'polyethylene_glycol_3350+electrolytes',
    'peg-3350+nacl+na_bicarbon': 'polyethylene_glycol_3350+electrolytes',
    'peg-3350+sodium_sulf+nacl': 'polyethylene_glycol_3350+electrolytes',
    'peg-3350+sodium_chloride+na_bicarbon': 'polyethylene_glycol_3350+electrolytes',
    'peg-3350+sodium_sulf+sodium_chloride': 'polyethylene_glycol_3350+electrolytes',
    'peg_3350': 'polyethylene_glycol_3350',
    'peg_3350+electrolytes': 'polyethylene_glycol_3350+electrolytes',
    'peg3350': 'polyethylene_glycol_3350',
    'piqray_200mg_daily_dose': 'piqray',
    'piqray_250mg_daily_dose': 'piqray',
    'piqray_300mg_daily_dose': 'piqray',
    'pirmella_1/35': 'pirmella',
    'pirmella_7/7/7': 'pirmella',
    'pneumovax_23': 'pneumococcal_vaccine',
    'pneumovax_23/1_dose': 'pneumococcal_vaccine',
    'pneumovax_23/5_dose': 'pneumococcal_vaccine',
    'potassium_chloride+dextro': 'potassium_chloride+dextrose',
    'potassium_chloride+sodium': 'potassium_chloride+sodium_chloride',
    'potassium_chloride_0.15%+d5w+sodium_chloride_0.45%': 'potassium_chloride+dextrose+sodium_chloride',
    'potassium_chloride_0.15%+d5w+sodium_chloride_0.9%': 'potassium_chloride+dextrose+sodium_chloride',
    'potassium_chloride_0.3%+d5w+sodium_chloride_0.45%': 'potassium_chloride+dextrose+sodium_chloride',
    'potassium_chloride_cr': 'potassium_chloride+cr',
    'potassium_chloride_er': 'potassium_chloride+er',
    'potassium_chloride_sr': 'potassium_chloride+sr',
    'prednisolone_acetate': 'prednisolone',
    'prednisolone_acetate_p-f': 'prednisolone',
    'prednisolone_sodium_phosp': 'prednisolone',
    'prednisolonee_sodium_phosp': 'prednisolone',
    'prednisolonee': 'prednisolone',
    'prednisone': 'prednisone',
    'prednisone_intensol': 'prednisone'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [188]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_p)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [189]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'p'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("p"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|   drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+------------+----+---------------+----------------------+
|   1003816184|2020-03-17|2020-04-15|       p & s|2020|              0|                 p_&_s|
|   1011005745|2020-11-15|2020-12-02|       p & s|2020|              0|                 p_&_s|
|   1011005745|2019-03-25|2019-04-16|       p & s|2019|              0|                 p_&_s|
|   1011005745|2020-07-23|2020-09-02|       p & s|2020|              0|                 p_&_s|
|   1003816184|2020-03-17|2020-04-17|       p & s|2020|              0|                 p_&_s|
|   1011005745|2020-09-02|2020-09-02|       p & s|2020|              0|                 p_&_s|
|   1011005745|2019-03-21|2019-04-16|       p & s|2019|              0|                 p_&_s|
|   1003816184|2020-03-17|2020-04-20|       p & s|

In [190]:
# Q drug names

consolidations_q = {
    'quetiapine_fumarate': 'quetiapine',
    'quetiapine_fumarate_er': 'quetiapine+er',
    'quinapril_hydrochloride': 'quinapril',
    'quinapril_hcl': 'quinapril',
    'qnasl': 'beclomethasone',
    'quartette': 'levonorgestrel+ethinyl_estradiol',
    'quasense': 'levonorgestrel+ethinyl_estradiol',
    'quillichew_er': 'methylphenidate+er',
    'quillivant_xr': 'methylphenidate+er',
    'quinidine_sulfate': 'quinidine',
    'quinine_sulfate': 'quinine',
    'qvar': 'beclomethasone',
    'qvar_redihaler': 'beclomethasone'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [191]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_q)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [192]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'q'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("q"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+---------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+---------+----+---------------+----------------------+
|   1007583814|2017-09-25|2017-10-02| q-bid-dm|2017|              0|              q-bid-dm|
|   1004672506|2016-09-11|2016-10-10|   q-dryl|2016|              0|                q-dryl|
|   1000491134|2017-07-10|2017-07-21|   q-dryl|2017|              0|                q-dryl|
|   1005760258|2016-02-13|2016-02-13|   q-dryl|2016|              0|                q-dryl|
|   1002949821|2018-10-13|2018-10-13|   q-dryl|2018|              0|                q-dryl|
|   1004672506|2016-09-11|2016-10-21|   q-dryl|2016|              0|                q-dryl|
|   1000491134|2017-07-10|2017-07-10|   q-dryl|2017|              0|                q-dryl|
|   1001005054|2016-01-25|2016-02-02|   q-dryl|2016|              0|            

In [193]:
# R drug names

consolidations_r = {
    'raloxifene_hydrochloride': 'raloxifene',
    'ranitidine_hydrochloride': 'ranitidine',
    'reclast': 'zoledronic_acid',
    'regenecare_ha': 'lidocaine',
    'relion_ultra_thin_lancets': 'not_drug',
    'repatha': 'evolocumab',
    'repatha_sureclick': 'evolocumab',
    'restasis': 'cyclosporine_ophthalmic',
    'restasis_multidose': 'cyclosporine_ophthalmic',
    'revlimid': 'lenalidomide',
    'rexulti': 'brexpiprazole',
    'reyataz': 'atazanavir',
    'ribasphere': 'ribavirin',
    'risedronate_sodium': 'risedronate',
    'risedronate_sodium_dr': 'risedronate',
    'rivastigmine_transdermal': 'rivastigmine',
    'rosuvastatin_calcium': 'rosuvastatin',
    'rozerem': 'ramelteon',
    'rydapt': 'midostaurin'
}


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [194]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_r)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [195]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'r'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("r"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop sort priority column to use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+---------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+---------+----+---------------+----------------------+
|   1003027291|2019-10-23|2019-11-13|r-gene 10|2019|              0|             r-gene_10|
|   1021438694|2016-12-12|2016-12-15|r-gene 10|2016|              0|             r-gene_10|
|   1021438694|2016-12-12|2017-01-17|r-gene 10|2017|              0|             r-gene_10|
|   1012609248|2017-04-07|2017-05-01|r-gene 10|2017|              0|             r-gene_10|
|   1001968675|2019-11-11|2019-11-11|r-gene 10|2019|              0|             r-gene_10|
|   1012609248|2017-04-07|2017-04-07|r-gene 10|2017|              0|             r-gene_10|
|   1000733677|2020-09-09|2020-09-09|r-gene 10|2020|              0|             r-gene_10|
|   1001968675|2019-11-11|2019-11-14|r-gene 10|2019|              0|            

In [196]:
# S drug names

consolidations_s = {
    'sertraline_hcl': 'sertraline',
    'sumatriptan_succinate': 'sumatriptan',
    'symbicort': 'budesonide+formoterol',
    'synthroid': 'levothyroxine',
    'senna': 'sennosides',
    'senna_plus': 'sennosides+docusate',
    'senna_s': 'sennosides+docusate',
    'sevelamer_carbonate': 'sevelamer',
    'sevelamer_hcl': 'sevelamer',
    'sildenafil_citrate': 'sildenafil',
    'sitagliptin_phosphate': 'sitagliptin',
    'solifenacin_succinate': 'solifenacin',
    'sotalol_hcl': 'sotalol',
    'spiriva': 'tiotropium',
    'sumatriptan_succinate': 'sumatriptan',
    'symbicort': 'budesonide+formoterol'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [197]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_s)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [198]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 's'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("s"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+--------------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|           drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+--------------------+----+---------------+----------------------+
|   1007505161|2019-04-24|2019-04-24|sm skin cleanser ...|2019|              0|  s+sm_skin_cleanse...|
|   1016048111|2016-06-17|2016-06-23|sm skin cleanser ...|2016|              0|  s+sm_skin_cleanse...|
|   1016048111|2016-06-20|2016-06-23|sm skin cleanser ...|2016|              0|  s+sm_skin_cleanse...|
|   1016048111|2016-06-17|2016-06-17|sm skin cleanser ...|2016|              0|  s+sm_skin_cleanse...|
|   1000880001|2018-09-28|2018-09-28|                  s2|2018|              0|                    s2|
|   1005982794|2017-01-16|2017-01-16|                  s2|2017|              0|                    s2|
|   1003760589|2019-08-24|2019-08-30|                  s2|2019|          

In [199]:
# T drug names

consolidations_t = {
    'tamsulosin_hydrochloride': 'tamsulosin',
    'testosterone_cypionate': 'testosterone',
    'tiotropium_bromide': 'tiotropium',
    'topiramate_er': 'topiramate+er',
    'tramadol_hcl': 'tramadol',
    'trazodone_hydrochloride': 'trazodone',
    'triamcinolone_acetonide': 'triamcinolone',
    'toujeo_solostar': 'insulin_glargine',
    'travatan_z': 'travoprost',
    'tresiba': 'insulin_degludec',
    'trulicity': 'dulaglutide',
    'tysabri': 'natalizumab'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [200]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_t)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [201]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 't'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("t"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+--------------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|           drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+--------------------+----+---------------+----------------------+
|   1005395028|2016-11-26|2017-01-07|t:slim g4 insulin...|2017|              0|  t:slim_g4_insulin...|
|   1005395028|2016-08-29|2016-09-22|t:slim g4 insulin...|2016|              0|  t:slim_g4_insulin...|
|   1000199497|2016-02-19|2016-03-14|t:slim g4 insulin...|2016|              0|  t:slim_g4_insulin...|
|   1005395028|2016-11-26|2017-01-05|t:slim g4 insulin...|2017|              0|  t:slim_g4_insulin...|
|   1000199497|2016-02-19|2016-03-08|t:slim g4 insulin...|2016|              0|  t:slim_g4_insulin...|
|   1000199497|2016-07-11|2016-08-03|t:slim g4 insulin...|2016|              0|  t:slim_g4_insulin...|
|   1000199497|2016-03-18|2016-04-06|t:slim g4 insulin...|2016|          

In [202]:
# U drug names

consolidations_u = {
    'unifine_safecontrol_pen_n': 'not_drug',
    'ulticare_short_pen_needle': 'not_drug',
    'ultra-care_lancets_30g': 'not_drug',
    'unilet_lancets_micro-thin': 'not_drug',
    'unistik_czt_normal': 'not_drug',
    'ultra-care_alcohol_prep_p': 'not_drug',
    'ultilet_lancets': 'not_drug',
    'unisol_4': 'unisol'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [203]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_u)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [204]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'u'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("u"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+---------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+---------+----+---------------+----------------------+
|   1006358716|2016-03-18|2016-03-18|   u-cort|2016|              0|                u-cort|
|   1006358716|2016-03-18|2016-04-27|   u-cort|2016|              0|                u-cort|
|   1006358716|2016-03-18|2016-04-05|   u-cort|2016|              0|                u-cort|
|   1006358716|2016-03-18|2016-04-07|   u-cort|2016|              0|                u-cort|
|   1006358716|2016-03-18|2016-04-19|   u-cort|2016|              0|                u-cort|
|   1006358716|2016-03-18|2016-04-29|   u-cort|2016|              0|                u-cort|
|   1008560349|2018-03-05|2018-04-03|ubiquinol|2018|              0|             ubiquinol|
|   1006015978|2018-03-01|2018-03-05|ubiquinol|2018|              0|            

In [205]:
# V drug names

consolidations_v = {
    'valtrex': 'valacyclovir',
    'vancomycin_hcl': 'vancomycin',
    'vascepa': 'icosapent_ethyl',
    'venlafaxine_hcl': 'venlafaxine',
    'venlafaxine_hcl_er': 'venlafaxine+er',
    'ventolin_hfa': 'albuterol',
    'verapamil_hcl': 'verapamil',
    'verapamil_hcl_er': 'verapamil+er',
    'vesicare': 'solifenacin',
    'viagra': 'sildenafil',
    'victoza': 'liraglutide',
    'viibryd': 'vilazodone',
    'vitamin_b-12': 'vitamin_b12',
    'vitamin_b12': 'vitamin_b12',
    'vitamin_d': 'vitamin_d',
    'vitamin_d3': 'vitamin_d3',
    'vitekta': 'elvitegravir',
    'vistaril': 'hydroxyzine',
    'vit_d': 'vitamin_d',
    'voltaren': 'diclofenac',
    'voriconazole': 'voriconazole',
    'vraylar': 'cariprazine',
    'vytorin': 'ezetimibe+simvastatin'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [206]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_v)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [207]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'v'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("v"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+--------------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|           drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+--------------------+----+---------------+----------------------+
|   1004461259|2018-01-24|2018-03-09|v-r childrens non...|2018|              0|  v-r_childrens_non...|
|   1002574158|2018-04-20|2018-04-25|       v-r ear drops|2018|              0|         v-r_ear_drops|
|   1000008247|2018-06-04|2018-06-11|       v-r ear drops|2018|              0|         v-r_ear_drops|
|   1002574158|2018-04-20|2018-05-07|       v-r ear drops|2018|              0|         v-r_ear_drops|
|   1001239609|2018-02-26|2018-03-08|       v-r ear drops|2018|              0|         v-r_ear_drops|
|   1002574158|2018-04-20|2018-05-03|       v-r ear drops|2018|              0|         v-r_ear_drops|
|   1000008247|2018-06-04|2018-06-16|       v-r ear drops|2018|          

In [208]:
# W drug names

consolidations_w = {
    'warfarin_sodium': 'warfarin',
    'wellbutrin_sr': 'bupropion',
    'westhroid': 'thyroid',
    'wymzya_fe': 'ethinyl_estradiol+norethindrone+iron',
    'wal-fex_allergen': 'fexofenadine',
    'wal-phed': 'pseudoephedrine',
    'wal-itin_d_24_hour': 'loratadine+pseudoephedrine',
    'wal-tussin_cough': 'guaifenesin',
    'wal-tussin_cough_&_chest': 'guaifenesin',
    'wal-zan_75': 'ranitidine',
    'walgreens_glucose': 'glucose',
    'walgreens_ultra_thin_lanc': 'not_drug',
    'wp_thyroid': 'thyroid'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [209]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_w)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [210]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'w'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("w"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+---------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+---------+----+---------------+----------------------+
|   1003872992|2016-04-05|2016-04-22| wal-dram|2016|              0|              wal-dram|
|   1003872992|2016-04-05|2016-05-10| wal-dram|2016|              0|              wal-dram|
|   1003872992|2016-04-05|2016-04-05| wal-dram|2016|              0|              wal-dram|
|   1003872992|2016-04-05|2016-04-19| wal-dram|2016|              0|              wal-dram|
|   1003227736|2016-07-03|2016-08-04| wal-dryl|2016|              0|              wal-dryl|
|   1000589789|2018-04-17|2018-05-18| wal-dryl|2018|              0|              wal-dryl|
|   1003227736|2016-07-03|2016-08-08| wal-dryl|2016|              0|              wal-dryl|
|   1002709924|2019-05-03|2019-05-03| wal-dryl|2019|              0|            

In [211]:
# X drug names

consolidations_x = {
    'xigduo_xr': 'dapagliflozin+metformin',
    'xolair': 'omalizumab',
    'xarelto': 'rivaroxaban',
    'xanax': 'alprazolam',
    'xeljanz': 'tofacitinib',
    'xulane': 'ethinyl_estradiol+norelgestromin',
    'xarelto_starter_pack': 'rivaroxaban',
    'xofluza': 'baloxavir_marboxil',
    'xyzal': 'levocetirizine',
    'xtampza_er': 'oxycodone',
    'xeomin': 'incobotulinumtoxina',
    'xifaxan': 'rifaximin',
    'xylocaine': 'lidocaine',
    'xalatan': 'latanoprost',
    'xgeva': 'denosumab',
    'xofluza': 'baloxavir_marboxil',
    'xarelto_dose_pack': 'rivaroxaban',
    'xcopri': 'cenobamate',
    'xeljanz_xr': 'tofacitinib',
    'xyrem': 'sodium_oxybate',
    'xylocaine': 'lidocaine',
    'xylocaine-mpf': 'lidocaine',
    'xylocaine-mpf+epinephrine': 'lidocaine+epinephrine',
    'xylocaine_jelly': 'lidocaine_gel',
    'xylocaine+epinephrine': 'lidocaine+epinephrine'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [212]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_x)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [213]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'x'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("x"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+---------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+---------+----+---------------+----------------------+
|   1014241627|2016-04-19|2016-05-16|  x-viate|2016|              0|               x-viate|
|   1013126012|2016-10-24|2016-10-24|  x-viate|2016|              0|               x-viate|
|   1001492116|2016-04-19|2016-05-02|  x-viate|2016|              0|               x-viate|
|   1013126012|2016-10-24|2016-11-04|  x-viate|2016|              0|               x-viate|
|   1005131005|2016-12-02|2016-12-02|  x-viate|2016|              0|               x-viate|
|   1001492116|2016-04-19|2016-04-27|  x-viate|2016|              0|               x-viate|
|   1005131005|2016-12-02|2017-01-12|  x-viate|2017|              0|               x-viate|
|   1001492116|2016-04-19|2016-05-26|  x-viate|2016|              0|            

In [214]:
# Y drug names

consolidations_y = {
    'yaz': 'ethinyl_estradiol+drospirenone',
    'yasmin': 'ethinyl_estradiol+drospirenone',
    'yuvafem': 'estradiol',
    'yosprala': 'aspirin+omeprazole',
    'yondelis': 'trabectedin',
    'yohimbine': 'yohimbine',
    'yupelri': 'revefenacin'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [215]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_y)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [216]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'y'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("y"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+--------------------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|           drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+--------------------+----+---------------+----------------------+
|   1007041735|2020-01-23|2020-01-31|              y-tuss|2020|              0|                y-tuss|
|   1007041735|2020-01-23|2020-01-23|              y-tuss|2020|              0|                y-tuss|
|   1003679493|2016-06-05|2016-07-12|              yf-vax|2016|              0|                yf-vax|
|   1015659454|2017-05-19|2017-05-25|              yf-vax|2017|              0|                yf-vax|
|   1003467651|2018-06-28|2018-08-08|your life multi a...|2018|              0|  your_life_multi_a...|
|   1006994438|2020-04-15|2020-05-15|your life multi a...|2020|              0|  your_life_multi_a...|
|   1003467651|2018-06-28|2018-07-24|your life multi a...|2018|          

In [217]:
# Z drug names

consolidations_z = {
    'zenpep': 'pancrelipase',
    'zithromax': 'azithromycin',
    'zolpidem_tartrate': 'zolpidem',
    'zyprexa': 'olanzapine',
    'zylet': 'lotepraxol+tobramycin',
    'zerbaxa': 'ceftolozane+tazobactam',
    'zyloprim': 'allopurinol',
    'zirgan': 'ganciclovir_ophthalmic',
    'ziprasidone_hcl': 'ziprasidone',
    'zolpimist': 'zolpidem',
    'zostrix': 'capsaicin',
    'zofran': 'ondansetron',
    'zyrtec': 'cetirizine',
    'zantac': 'ranitidine',
    'zetia': 'ezetimibe',
    'zovirax': 'acyclovir',
    'zocor': 'simvastatin',
    'zoloft': 'sertraline'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [218]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_d)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [219]:
from pyspark.sql.functions import asc

# Add a custom sort column that prioritizes names starting with 'z'
sorted_unique_levels = clean_df.withColumn(
    "sort_priority",
    when(col("standardized_drug_name").startswith("z"), 0).otherwise(1)
)

# Sort by the custom sort priority, and then alphabetically by 'standardized_drug_name'
sorted_unique_levels = sorted_unique_levels.orderBy(
    col("sort_priority"), 
    col("standardized_drug_name").asc()
)

# Drop priority sort column so we can use for new cases
sorted_unique_levels = sorted_unique_levels.drop("sort_priority")

# Show the sorted DataFrame
sorted_unique_levels.show(n=20)  # Adjust 'n' to display more or fewer results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+----------+---------+----+---------------+----------------------+
|mi_person_key| drug_date|  ADE_Date|drug_name|Year|Hospitalization|standardized_drug_name|
+-------------+----------+----------+---------+----+---------------+----------------------+
|   1003140408|2018-08-31|2018-08-31|    z-bum|2018|              0|                 z-bum|
|   1001455899|2020-05-26|2020-07-02|  z-clinz|2020|              0|               z-clinz|
|   1001455899|2020-05-26|2020-05-26|  z-clinz|2020|              0|               z-clinz|
|   1001076222|2018-03-30|2018-03-30|  zaditor|2018|              0|               zaditor|
|   1019023358|2016-03-18|2016-04-26|  zaditor|2016|              0|               zaditor|
|   1001580784|2018-07-24|2018-08-01|  zaditor|2018|              0|               zaditor|
|   1007907253|2018-04-25|2018-04-25|  zaditor|2018|              0|               zaditor|
|   1001580784|2018-09-17|2018-09-28|  zaditor|2018|              0|            

In [220]:
# Not drugs

consolidations_not_drugs = {
    '3ml_luer-lok_syringe': 'not_drug',
    'bd_insulin_syringe': 'not_drug',
    'truedraw_lancing_device': 'not_drug',
    'kroger_lancing_device': 'not_drug',
    'alcohol_prep_pads': 'not_drug',
    'lancets': 'not_drug',
    'catheters': 'not_drug',
    'cvs_moisturizing_cream': 'not_drug',
    'hrt_cream_base': 'not_drug',
    'systane_gel': 'not_drug',
    'arnica_gel': 'not_drug',
    'cvs_moisturizing_lotion': 'not_drug',
    'salicylic_acid_lotion': 'not_drug',
    'silverseal_hydrogel_dress': 'not_drug',
    'carrasyn_hydrogel_wound_d': 'not_drug',
    'ra_saline_nasal_spray': 'not_drug',
    'little_noses_saline': 'not_drug',
    'refresh_liquigel': 'not_drug',
    'eye_irrigating_solutions': 'not_drug',
    'control_solutions_for_glucose_meters': 'not_drug',
    'vaginal_cream_applicators': 'not_drug',
    'silica_gel_packets': 'not_drug',
    'alcohol_prep_pads': 'not_drug',
    'prodigy_twist_top_lancets': 'not_drug',
    'alcohol_swabs': 'not_drug',
    'sm_alcohol_prep_pads': 'not_drug',
    'sterile_water_for_irrigat': 'not_drug',
    'dexcom_g5_mobile_transmit': 'not_drug',
    'easy_touch_32gx6mm': 'not_drug',
    'ulticare_short_pen_needle': 'not_drug',
    'global_inject_ease_insuli': 'not_drug',
    'e-z-disk': 'not_drug',
    'accu-chek_guide': 'not_drug',
    'true_metrix': 'not_drug',
    't:flex': 'not_drug',
    't:slim': 'not_drug',
    'prodigy_pocket_no_match_gluc': 'not_drug',
    'compact_space_chamber/ant': 'not_drug',
    'onetouch_suresoft_lancing': 'not_drug',
    'relion_ultra_thin_lancets': 'not_drug',
    'bd_pen_needle/mini/ultra-': 'not_drug',
    'accu-chek_fastclix_lancet': 'not_drug',
    'adjustable_lancing_device': 'not_drug',
    'bd_1ml_tuberculin_syringe': 'not_drug',
    'novofine_autocover_pen_ne': 'not_drug',
    'unifine_pentips_plus_31gx': 'not_drug',
    'optichamber_diamond': 'not_drug',
    'bayer_microlet_lancets': 'not_drug',
    'onetouch_delica_lancets_e': 'not_drug',
    'trueplus_lancets_30g_ultr': 'not_drug',
    'freestyle_libre/sensor/fl': 'not_drug',
    'relion_pen_needles/31g_x': 'not_drug',
    'bd_pen_needle/nano_2nd_ge': 'not_drug',
    'ulticare_insulin_syringe/': 'not_drug',
    'accu-chek_aviva': 'not_drug'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [221]:
clean_df = replace_values_with_mapping(clean_df, 'standardized_drug_name', consolidations_not_drugs)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [222]:
from pyspark.sql.functions import col

# Group by drug name and get counts
unique_levels_clean = clean_df.groupBy("standardized_drug_name").count()

# Order by count in descending order
unique_levels_clean = unique_levels_clean.orderBy(col("count").desc())

# Show the sorted results
unique_levels_clean.show(50)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+-------+
|standardized_drug_name|  count|
+----------------------+-------+
|           amoxicillin|1968898|
|             albuterol|1404116|
|           montelukast| 924671|
|  clonidine_hydroch...| 845305|
|       methylphenidate| 833450|
|            guanfacine| 692627|
|              cefdinir| 655702|
|          prednisolone| 653299|
|            cetirizine| 607289|
|          azithromycin| 563921|
|               vyvanse| 531135|
|  amoxicillin+clavu...| 522423|
|  fluticasone_propi...| 500985|
|  amphetamine+dextr...| 492556|
|         triamcinolone| 416960|
|              nystatin| 344970|
|         guanfacine+er| 340888|
|       ondansetron+odt| 336166|
|            sertraline| 329458|
|              not_drug| 320230|
|        ranitidine_hcl| 309129|
|             mupirocin| 300741|
|           risperidone| 298291|
|            cephalexin| 289759|
|  polyethylene_glyc...| 281462|
|           flovent_hfa| 274922|
|           oseltamivir| 273164|
|        h

In [223]:
clean_df = clean_df.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [224]:
from pyspark.sql.functions import col

# Drop rows where standardized_drug_name is 'not_drug' or 'unknown'
clean_df = clean_df.filter(~col('standardized_drug_name').isin('not_drug', 'unknown'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [225]:
from pyspark.sql.functions import col

# Group by drug name and get counts
levels_unique = clean_df.groupBy("standardized_drug_name").count()

# Order by count in descending order
levels_unique = levels_unique.orderBy(col("count").desc())

# Show the sorted results
levels_unique.show(50)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+-------+
|standardized_drug_name|  count|
+----------------------+-------+
|           amoxicillin|1968898|
|             albuterol|1404116|
|           montelukast| 924671|
|  clonidine_hydroch...| 845305|
|       methylphenidate| 833450|
|            guanfacine| 692627|
|              cefdinir| 655702|
|          prednisolone| 653299|
|            cetirizine| 607289|
|          azithromycin| 563921|
|               vyvanse| 531135|
|  amoxicillin+clavu...| 522423|
|  fluticasone_propi...| 500985|
|  amphetamine+dextr...| 492556|
|         triamcinolone| 416960|
|              nystatin| 344970|
|         guanfacine+er| 340888|
|       ondansetron+odt| 336166|
|            sertraline| 329458|
|        ranitidine_hcl| 309129|
|             mupirocin| 300741|
|           risperidone| 298291|
|            cephalexin| 289759|
|  polyethylene_glyc...| 281462|
|           flovent_hfa| 274922|
|           oseltamivir| 273164|
|        hydrocortisone| 259357|
|         

In [226]:
# Reduce the number of partitions to optimize data collection
levels_unique = levels_unique.coalesce(1)  # Reduce to 1 partition

# Collect the results as a pandas DataFrame
unique_drug_names = levels_unique.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [227]:
import pandas as pd
from io import BytesIO
import boto3

# Initialize a boto3 client
s3_client = boto3.client('s3')

# Define S3 bucket and path
s3_bucket = "pgx-repository"
drug_name_s3_path = f"ade-risk-model/Step3_Normalize_Drug_Name/drug_names_cleaned_{cohort}.csv"

# Create a buffer
csv_buffer = BytesIO()
unique_drug_names.to_csv(csv_buffer, index=False)
csv_buffer.seek(0)

# Upload the buffer
s3_client.upload_fileobj(
    csv_buffer,
    s3_bucket,
    drug_name_s3_path
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [253]:
train_df = clean_df.filter((col("year") >= 2016) & (col("year") <= 2018)).distinct()
test_df = clean_df.filter(col("year") == 2019).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [254]:
# Get distinct mi_person_keys from both datasets
train_keys = train_df.select("mi_person_key").distinct()
test_keys = test_df.select("mi_person_key").distinct()

# Check for overlapping keys using a left anti join
overlap_keys = train_keys.join(test_keys, train_keys.mi_person_key == test_keys.mi_person_key, "left_anti")

# Count to see if there are any overlapping keys
overlap_count = overlap_keys.count()

if overlap_count == 0:
    print("No overlapping mi_person_keys between train and test datasets.")
else:
    print(f"Warning: There are {overlap_count} overlapping mi_person_keys between train and test datasets.")
    # Remove overlapping mi_person_keys from the test dataset using a left anti join
    test_df = test_df.join(train_keys, ["mi_person_key"], "left_anti")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [255]:
# # Calculate and show hospitalization counts for both dataframes
def count_values(df, column_name):
    # Count the occurrences of each value in the specified column
    count_df = df.groupBy(column_name).count()
    # Show the result
    count_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [256]:
print("Counts for Training Dataset:")
count_values(train_df, "hospitalization")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Counts for Training Dataset:
+---------------+--------+
|hospitalization|   count|
+---------------+--------+
|              0|14896216|
|              1|    1399|
+---------------+--------+

In [257]:
print("Counts for Test Dataset:")
count_values(test_df, "hospitalization")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Counts for Test Dataset:
+---------------+-------+
|hospitalization|  count|
+---------------+-------+
|              0|1658051|
|              1|    131|
+---------------+-------+

In [258]:
train_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['mi_person_key', 'drug_date', 'ADE_Date', 'drug_name', 'Year', 'Hospitalization', 'standardized_drug_name']

In [259]:
test_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['mi_person_key', 'drug_date', 'ADE_Date', 'drug_name', 'Year', 'Hospitalization', 'standardized_drug_name']

In [260]:
from pyspark.sql.functions import col

# Convert mi_person_key to string and select specific columns for train and test sets
test_df = test_df.select(
    col("mi_person_key").cast("string").alias("mi_person_key"),
    "drug_date",
    "ADE_Date",
    "standardized_drug_name",
    "Hospitalization"
)

train_df = train_df.select(
    col("mi_person_key").cast("string").alias("mi_person_key"),
    "drug_date",
    "ADE_Date",
    "standardized_drug_name",
    "Hospitalization"
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [261]:
train_df = train_df.withColumnRenamed("Hospitalization", "label")
test_df = test_df.withColumnRenamed("Hospitalization", "label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [262]:
print("Train DataFrame Schema:")
train_df.printSchema()
print("\nTest DataFrame Schema:")
test_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Train DataFrame Schema:
root
 |-- mi_person_key: string (nullable = true)
 |-- drug_date: date (nullable = true)
 |-- ADE_Date: date (nullable = true)
 |-- standardized_drug_name: string (nullable = true)
 |-- label: string (nullable = false)


Test DataFrame Schema:
root
 |-- mi_person_key: string (nullable = true)
 |-- drug_date: date (nullable = true)
 |-- ADE_Date: date (nullable = true)
 |-- standardized_drug_name: string (nullable = true)
 |-- label: string (nullable = false)

In [263]:
train_df.cache()
test_df.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[mi_person_key: string, drug_date: date, ADE_Date: date, standardized_drug_name: string, label: string]

In [265]:
# Paths for saving datasets to S3
s3_bucket = f"s3://pgx-repository/ade-risk-model/Step5_Time_to_Event_Model/1_input_datasets/{cohort}"
train_output_path = f"{s3_bucket}/train"
test_output_path = f"{s3_bucket}/test"

# Save train and test DataFrames to S3 in Parquet format
train_df.write.mode("overwrite").parquet(train_output_path)
test_df.write.mode("overwrite").parquet(test_output_path)

print(f"Train dataset saved to {train_output_path}")
print(f"Test dataset saved to {test_output_path}")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Train dataset saved to s3://pgx-repository/ade-risk-model/Step5_Time_to_Event_Model/1_input_datasets/cohort1/train
Test dataset saved to s3://pgx-repository/ade-risk-model/Step5_Time_to_Event_Model/1_input_datasets/cohort1/test

In [266]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

# Step 1: StringIndexer for mi_person_key (categorical feature)
person_key_indexer = StringIndexer(inputCol="mi_person_key", outputCol="person_key_index", handleInvalid="keep")

# Step 2: StringIndexer for standardized_drug_name
drug_name_indexer = StringIndexer(inputCol="standardized_drug_name", outputCol="drug_name_index", handleInvalid="keep")

# Step 3: OneHotEncoder for drug_name_index (leave mi_person_key as indexed)
one_hot_encoder = OneHotEncoder(inputCol="drug_name_index", outputCol="drug_name_one_hot")

# Step 4: VectorAssembler to combine features (include person_key_index)
assembler = VectorAssembler(
    inputCols=["person_key_index", "drug_name_one_hot"],  # Include mi_person_key as an indexed categorical feature
    outputCol="features"
)

# Create a pipeline
pipeline = Pipeline(stages=[person_key_indexer, drug_name_indexer, one_hot_encoder, assembler])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [249]:
# Fit and transform the data
pipeline_model = pipeline.fit(train_df)

# Transform the dataset
processed_train_df = pipeline_model.transform(train_df)
processed_test_df = pipeline_model.transform(test_df)

# Inspect the transformed DataFrame
processed_train_df.select("mi_person_key", "person_key_index", "standardized_drug_name", "drug_name_index", "drug_name_one_hot", "features", "label").show(5, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------------+-------------------------+---------------+------------------+---------------------------+-----+
|mi_person_key|person_key_index|standardized_drug_name   |drug_name_index|drug_name_one_hot |features                   |label|
+-------------+----------------+-------------------------+---------------+------------------+---------------------------+-----+
|1000272491   |22377.0         |amoxicillin+clavulanate_p|7.0            |(5344,[7],[1.0])  |(5345,[0,8],[22377.0,1.0]) |0    |
|1000304459   |1910.0          |vyvanse                  |13.0           |(5344,[13],[1.0]) |(5345,[0,14],[1910.0,1.0]) |0    |
|1000406357   |2996.0          |mirtazapine              |128.0          |(5344,[128],[1.0])|(5345,[0,129],[2996.0,1.0])|0    |
|1000545101   |44217.0         |vyvanse                  |13.0           |(5344,[13],[1.0]) |(5345,[0,14],[44217.0,1.0])|0    |
|1000644696   |1962.0          |cvs_childrens_allergy    |169.0          |(5344,[169],[1.0])|(5345,[0,17

In [268]:
import json
import boto3
import io

# Paths for saving processed datasets to S3
s3_bucket = f"s3://pgx-repository/ade-risk-model/Step5_Time_to_Event_Model/2_processed_datasets/{cohort}"
processed_train_output_path = f"{s3_bucket}/train"
processed_test_output_path = f"{s3_bucket}/test"
feature_info_path = f"{s3_bucket}/feature_info_{cohort}.json"

# Save train and test DataFrames to S3 in Parquet format
processed_train_df.write.mode("overwrite").parquet(processed_train_output_path)
processed_test_df.write.mode("overwrite").parquet(processed_test_output_path)

print(f"Processed Train dataset saved to {processed_train_output_path}")
print(f"Processed Test dataset saved to {processed_test_output_path}")

# Extract feature names from StringIndexer
index_to_category = {i: category for i, category in enumerate(pipeline_model.stages[1].labels)}
feature_names = [
    f"{category} | drug_name_index_{i}"  # Use a separator (e.g., ->) here
    for i, category in index_to_category.items()
]

# Save feature names and types
feature_info = {
    "names": feature_names,
    "types": "one-hot encoded vector",
    "source_column": "drug_name"
}

# Save the JSON to S3 using an in-memory buffer
bucket_name = s3_bucket.split('/')[2]
s3_key = "/".join(s3_bucket.split('/')[3:]) + f"/feature_info_{cohort}.json"

buffer = io.BytesIO()
buffer.write(json.dumps(feature_info, indent=4).encode('utf-8'))
buffer.seek(0)

s3_client.upload_fileobj(buffer, bucket_name, s3_key)

print(f"Feature information for {cohort} saved to {feature_info_path}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Processed Train dataset saved to s3://pgx-repository/ade-risk-model/Step5_Time_to_Event_Model/2_processed_datasets/cohort1/train
Processed Test dataset saved to s3://pgx-repository/ade-risk-model/Step5_Time_to_Event_Model/2_processed_datasets/cohort1/test
Feature information for cohort1 saved to s3://pgx-repository/ade-risk-model/Step5_Time_to_Event_Model/2_processed_datasets/cohort1/feature_info_cohort1.json